classification task on the MNIST dataset.

In [1]:
pip install mlflow

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install keras

^C
Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.2 MB ? eta -:--:--
   ----------------- ---------------------- 0.5/1.2 MB 1.5 MB/s eta 0:00:01
   -------------------------- ------------- 0.8/1.2 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
   ------- -------------------------------- 0.5/3.0 MB 985.5 kB/s eta 0:00:03
   ---------- ----------------------------- 0.8/3.0 MB 931.2 kB/s eta 0:00:03
   -------------- ------------------------- 1.0/3.0 MB 1.0 MB/s eta 0:00:02
   ----------------- ---------------------- 1.3/3.0 MB 1.1 MB/s eta 0:00:02
   ------------------------ --------------- 1.8/3.0 MB 1.2 MB/s eta 0:00:01
   ---------------------------- -----------

In [2]:
# Import necessary libraries
import mlflow
import mlflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
import os

ModuleNotFoundError: No module named 'keras'

In [ ]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize the data to values between 0 and 1
X_train, X_test = X_train / 255.0, X_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Set hyperparameters
learning_rate = 0.001
epochs = 10
batch_size = 32
num_classes = 10

In [ ]:
# Define the FFNN model
def create_ffnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))  # Flatten the input images (28x28) to a 1D vector
    model.add(Dense(128, activation='relu'))     # Fully connected layer with 128 neurons
    model.add(Dense(64, activation='relu'))      # Fully connected layer with 64 neurons
    model.add(Dense(num_classes, activation='softmax'))  # Output layer with softmax for classification
    return model

In [ ]:
# Start an MLflow run
with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("batch_size", batch_size)

    # Create and compile the model
    model = create_ffnn_model(input_shape=(28, 28), num_classes=num_classes)
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss=SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    # Train the model and log the metrics for each epoch
    for epoch in range(epochs):
        history = model.fit(X_train, y_train, batch_size=batch_size, epochs=1, validation_data=(X_test, y_test), verbose=0)

        # Get accuracy and loss from history and log them
        train_loss = history.history['loss'][0]
        train_acc = history.history['accuracy'][0]
        val_loss = history.history['val_loss'][0]
        val_acc = history.history['val_accuracy'][0]

        # Log metrics
        mlflow.log_metric("train_loss", train_loss, step=epoch)
        mlflow.log_metric("train_accuracy", train_acc, step=epoch)
        mlflow.log_metric("val_loss", val_loss, step=epoch)
        mlflow.log_metric("val_accuracy", val_acc, step=epoch)

        print(f"Epoch {epoch+1}/{epochs} - Train Accuracy: {train_acc:.4f}, Validation Accuracy: {val_acc:.4f}")

    # Save the model as an artifact
    model_filename = "ffnn_mnist_model.h5"
    model.save(model_filename)
    mlflow.log_artifact(model_filename)

    # Log an additional artifact (e.g., model summary)
    with open("model_summary.txt", "w") as f:
        model.summary(print_fn=lambda x: f.write(x + '\n'))
    mlflow.log_artifact("model_summary.txt")

    print("Run complete. Check MLflow UI for details.")

# Instructions to view the results in MLflow UI:
# After running the script, type "mlflow ui" in your terminal to start the UI, and navigate to http://localhost:5000


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10 - Train Accuracy: 0.9305, Validation Accuracy: 0.9620
Epoch 2/10 - Train Accuracy: 0.9696, Validation Accuracy: 0.9676
Epoch 3/10 - Train Accuracy: 0.9778, Validation Accuracy: 0.9729
Epoch 4/10 - Train Accuracy: 0.9835, Validation Accuracy: 0.9750
Epoch 5/10 - Train Accuracy: 0.9866, Validation Accuracy: 0.9719
Epoch 6/10 - Train Accuracy: 0.9880, Validation Accuracy: 0.9747
Epoch 7/10 - Train Accuracy: 0.9909, Validation Accuracy: 0.9783
Epoch 8/10 - Train Accuracy: 0.9924, Validation Accuracy: 0.9783
Epoch 9/10 - Train Accuracy: 0.9931, Validation Accuracy: 0.9775


Epoch 10/10 - Train Accuracy: 0.9936, Validation Accuracy: 0.9778


Run complete. Check MLflow UI for details.
